In [ ]:
!pip install playwright

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 16.0 MB/s eta 0:00:00


In [ ]:
! playwright install

171.6 MiB [] 0% 0.0s171.6 MiB [] 0% 55.4s171.6 MiB [] 0% 28.8s171.6 MiB [] 0% 18.3s171.6 MiB [] 0% 10.2s171.6 MiB [] 1% 6.4s171.6 MiB [] 2% 4.9s171.6 MiB [] 2% 4.2s171.6 MiB [] 3% 3.6s171.6 MiB [] 4% 3.3s171.6 MiB [] 5% 3.1s171.6 MiB [] 5% 3.3s171.6 MiB [] 5% 3.4s171.6 MiB [] 6% 3.5s171.6 MiB [] 6% 3.6s171.6 MiB [] 7% 3.6s171.6 MiB [] 7% 3.7s171.6 MiB [] 8% 3.7s171.6 MiB [] 8% 3.8s171.6 MiB [] 9% 3.8s171.6 MiB [] 9% 3.9s171.6 MiB [] 9% 4.0s171.6 MiB [] 10% 4.0s171.6 MiB [] 10% 4.1s171.6 MiB [] 11% 4.1s171.6 MiB [] 11% 4.2s171.6 MiB [] 12% 4.3s171.6 MiB [] 13% 4.1s171.6 MiB [] 13% 4.0s171.6 MiB [] 14% 3.9s171.6 MiB [] 14% 3.8s171.6 MiB [] 15% 3.7s171.6 MiB [] 15% 3.6s171.6 MiB [] 16% 3.6s171.6 MiB [] 16% 3.7s171.6 MiB [] 16% 3.8s171.6 MiB [] 17% 3.6s171.6 MiB [] 18% 3.6s171.6 MiB [] 18% 3.5s171.6 MiB [] 19% 3.4s171.6 MiB [] 20% 3.4s171.6 MiB [] 21% 3.4s171.6 MiB [] 21% 3.5s171.6 MiB [] 22% 3.5s171.6 MiB [] 23% 3.4s171.6 MiB [] 24% 3.3s171.6 MiB [] 25% 3.1s171.6 MiB [] 26% 3.0s171.6 MiB 

**SCRAPPING TEXT**

In [ ]:
from playwright.async_api import async_playwright
import asyncio

In [ ]:
content = None
async def main():
    async with async_playwright() as p:
      global content
      print("Launching browser...")
      browser = await p.chromium.launch(headless=True)
      context = await browser.new_context(user_agent=(
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
      ))
      page = await context.new_page()

      print("Navigating to page...")
      await page.goto(
            'https://en.wikisource.org/wiki/The_Gates_of_Morning/Book_1/Chapter_1',
            wait_until='domcontentloaded',
            timeout=10000
        )

      print("Getting Content: ")
      paragraphs = page.locator("div.mw-parser-output p")
      count = await paragraphs.count()

      content_list = []
      for i in range(count):
          text = await paragraphs.nth(i).inner_text()
          if text.strip():
              content_list.append(text.strip())

      content = ' '.join(content_list)

      await browser.close()
      print("Done.")

await main()

Launching browser...
Navigating to page...
Getting Content: 
Done.


In [ ]:
!pip install langgraph langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 11.5 MB/s eta 0:00:00



**CREATE WORKFLOWS AND CHAIN**


In [ ]:
import os
from langchain.chains import LLMChain
from langchain import PromptTemplate
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, END

In [ ]:
os.environ["GROQ_API_KEY"] = "apikey"
llm = ChatGroq(groq_api_key=os.environ["GROQ_API_KEY"], model_name="llama3-70b-8192")

In [ ]:
spinner_prompt = PromptTemplate(
    input_variables=['content'],
    template='''
You are BookSpin AI, an expert literary transformation specialist. Your role is to take existing books, stories, or written content and create
compelling variations while maintaining the essence of the original work. You excel at reimagining narratives through different lenses, perspectives, genres,
 and styles. Dont give any other explanations, output the spinned book alone The content of the book you need to spin or rewrite is
 {content}
'''
)
spinner_chain = LLMChain(llm=llm, prompt=spinner_prompt)

reviewer_prompt = PromptTemplate(
    input_variables=['spinned_content'],
    template='''llm
    You are BookSpin Reviewer AI, a meticulous literary quality analyst specializing in reviewing rewritten or transformed narratives. Your role is to carefully analyze the rewritten version of a book, story, or content created by another AI (BookSpin AI), and provide constructive feedback on the following aspects:

- Faithfulness to the original core message, themes, and emotional tone.
- Creativity in narrative transformation (style, voice, genre shifts).
- Clarity, coherence, and readability of the rewritten content.
- Grammar, sentence structure, and literary flow.

Provide a clear review that includes:
1. A brief summary of what was done well.
2. Specific feedback on what could be improved.
3. Direct textual corrections or rewrites where necessary.

Do not repeat the original or rewritten content. Focus only on evaluation and improvement suggestions.

The spinned (rewritten) content to review is:
{spinned_content}


'''
)
reviewer_chain = LLMChain(llm=llm, prompt=reviewer_prompt)

editor_prompt = PromptTemplate(
    input_variables=['spinned_content', 'review_feedback'],
    template='''You are BookSpin Editor AI, a highly skilled literary editor trained to revise and enhance rewritten books or stories based on expert feedback. You are given two inputs:

1. The spinned content: a rewritten version of an original story, generated by another AI.
2. A detailed review and set of improvement suggestions provided by a literary reviewer AI.

Your task is to:
- Incorporate the feedback and corrections provided by the reviewer.
- Improve the content without changing the original transformation's creative direction.
- Preserve the artistic tone, narrative voice, and thematic essence of the spinned version.
- Make only necessary edits to grammar, style, coherence, and alignment with the original message.

Output only the **final edited version** of the spinned content. Do not include any explanations or references to the review or original version.

Spinned Content:
{spinned_content}

Review and Suggestions:
{review_feedback}
'''
)
editor_chain = LLMChain(llm=llm, prompt=editor_prompt)


In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 548.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 6.5 MB/s eta

**NEW COLLECTION**

In [ ]:
import chromadb

collection_name = input("Enter the name of the book (collection) if you are spinning new book: ")

chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection(name=collection_name)

Enter the name of the book (collection) if you are spinning new book: Chapter-1


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


**AGENTS**

In [ ]:
def score_and_tag_generator(input):
  score_and_tag_generator_prompt = PromptTemplate(
      input_variables = ['input'],
      template ="""
Analyze the following text and perform two tasks:

1. Assign a quality score out of 10 based on:
   - Clarity
   - Coherence
   - Tone appropriateness
   - Content depth

2. Generate 2 to 4 quality tags that describe:
   - Tone (e.g., formal, creative)
   - Clarity (e.g., concise, verbose)
   - Content type (e.g., informative, poetic)
   - Style (e.g., factual, engaging)

TEXT:
{input}

Return your output strictly in the following format. Dont give anything extra. Not even a single character. Give only the score and the tags list separated by a comma. This is very strict
dont give the same score for all the inputs, and you can give more than four tags also.

<integer between 1 and 10>,tag1,tag2,tag3,tagns
"""
  )
  score_and_tag_generator_chain = LLMChain(llm=llm,prompt=score_and_tag_generator_prompt)

  score_tag = score_and_tag_generator_chain.run(input=input)


  s = score_tag


  score_str, tags_str = s.split(',', 1)

  score = int(score_str.strip())
  tags = tags_str.strip()
  return score,tags

In [ ]:
from typing import TypedDict, Optional
from datetime import datetime
import ast


class BookSpinState(TypedDict):
    book: str
    spinned: Optional[str]
    review: Optional[str]
    final: Optional[str]


def edit_with_user(prompt_msg, default_value):
    print(prompt_msg)
    print("-" * 40)
    print(default_value)
    print("-" * 40)
    print("You can now edit the content. Leave blank space to accept without any changes.")
    user_edit = input("Enter your edited content:")
    return user_edit.strip() if user_edit.strip() else default_value


def spin_node(state: BookSpinState):
  ai_result = spinner_chain.run(content=state["book"])
  user_approved = edit_with_user("Spinned Output", ai_result)
  score,tags = score_and_tag_generator(user_approved)


  collection.add(
  documents=[content],
  ids=[f"v_{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}"],
  metadatas=[{
      "version":"spinned",
      "source":"wikisource",
      "generator_agent":"spin",
      "timestamp":datetime.utcnow().isoformat(),
      "score": score,
      "quality_tags": tags,
      "length":len(user_approved),
    }])

  return {"spinned": user_approved}

def review_node(state: BookSpinState):
    ai_result = reviewer_chain.run(spinned_content=state["spinned"])
    user_approved = edit_with_user("Review Feedback", ai_result)
    return {"review": user_approved}

def edit_node(state: BookSpinState):
  ai_result = editor_chain.run(
      spinned_content=state["spinned"],
      review_feedback=state["review"]
  )
  user_approved = edit_with_user("Final Edited Version", ai_result)
  score,tags = score_and_tag_generator(user_approved)

  collection.add(
  documents=[content],
  ids=[f"v_{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}"],
  metadatas=[{
      "version":"spinned_final",
      "source":"wikisource",
      "generator_agent":"editor",
      "timestamp":datetime.utcnow().isoformat(),
      "score": score,
      "quality_tags": tags,
      "length":len(user_approved),
    }])
  return {"final": user_approved}


**LANGGRAPH PART**

In [ ]:

builder = StateGraph(BookSpinState)

builder.add_node("spin", spin_node)
builder.add_node("review_node", review_node)
builder.add_node("edit_node", edit_node)

builder.set_entry_point("spin")
builder.add_edge("spin", "review_node")
builder.add_edge("review_node", "edit_node")
builder.add_edge("edit_node", END)

graph = builder.compile()

In [ ]:

final_state = graph.invoke({
    "book": content
})

print("\nWorkflow complete.")
print("\nFinal Edited Book:\n")
print(final_state["final"])



[HITL: Spinned Output]
----------------------------------------
Here is a rewritten version of "The Gates of Morning" with a focus on a fantasy setting and a strong female protagonist:

**The Shores of Dawn**

CHAPTER I: The Sea Witch's Isle

Kala stood on the windswept promontory, her emerald eyes fixed on the southern horizon. Behind her, the waves of the outer sea crashed against the rocky shores of her island, while before her, the calm waters of the lagoon shimmered like a sheet of silver. This was her domain, a realm of ancient magic, where the whispers of the sea goddesses still lingered in the wind.

As she gazed out at the lagoon, her thoughts turned to the people who called this place home. The women, children, and youths of the island were all hers to protect, and she had sworn to defend them against the darkness that lurked beyond the horizon. Her people were skilled in the arcane arts, and their canoes were crafted with magic and precision, but they were no match for the 

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given



[HITL: Review Feedback]
----------------------------------------
**BookSpin Reviewer AI's Evaluation**

**Overall Assessment:**
The rewritten version, "The Shores of Dawn," demonstrates a commendable effort in transforming the original narrative into a fantasy setting with a strong female protagonist. The story showcases a vivid and immersive world, with a clear sense of magic and wonder. However, there are areas that require attention to elevate the narrative to its full potential.

**What was done well:**

* The author has successfully created a captivating setting, with vivid descriptions of the island, the sea, and the lagoon.
* The introduction of Kala, the sea witch, is intriguing, and her backstory and motivations are skillfully woven throughout the chapter.
* The sense of foreboding and danger lurking beyond the horizon is effectively conveyed, creating tension and suspense.

**Specific Feedback on Improvement:**

* While the setting is richly described, some passages feel ove

In [ ]:
import chromadb
from pprint import pprint


results = collection.get(include=["documents", "metadatas"])

for i in range(len(results["ids"])):
    print(f"\nItem :{i+1}")
    print(f"ID:{results['ids'][i]}")
    print(f"Document:{results['documents'][i]}")
    print(f"Metadata:")
    pprint(results['metadatas'][i])
    print("-" * 60)



--- Item 1 ---
ID:         v_20250628_044804
Document:   "The Gates of Morning" CHAPTER I THE CANOE BUILDER D
ICK standing on a ledge of coral cast his eyes to the South. Behind him the breakers of the outer sea thundered and the spindrift scattered on the wind; before him stretched an ocean calm as a lake, infinite, blue, and flown about by the fishing gulls—the lagoon of Karolin. Clipped by its forty-mile ring of coral this great pond was a sea in itself, a sea of storm in heavy winds, a lake of azure, in light airs—and it was his—he who had landed here only yesterday. Women, children, youths, all the tribe to be seen busy along the beach in the blazing sun, fishing with nets, playing their games or working on the paraka patches, all were his people. His were the canoes drawn up on the sand and his the empty houses where the war canoes had once rested on their rollers. Then as he cast his eyes from the lagoon to the canoe houses his brow contracted, and, turning his back to the lago

In [ ]:
chroma_client.list_collections()


[Collection(name=the_gates_of_morning)]

In [ ]:
print(collection.get())

{'ids': ['v_20250628'], 'embeddings': None, 'documents': ['"The Gates of Morning" CHAPTER I THE CANOE BUILDER D\nICK standing on a ledge of coral cast his eyes to the South. Behind him the breakers of the outer sea thundered and the spindrift scattered on the wind; before him stretched an ocean calm as a lake, infinite, blue, and flown about by the fishing gulls—the lagoon of Karolin. Clipped by its forty-mile ring of coral this great pond was a sea in itself, a sea of storm in heavy winds, a lake of azure, in light airs—and it was his—he who had landed here only yesterday. Women, children, youths, all the tribe to be seen busy along the beach in the blazing sun, fishing with nets, playing their games or working on the paraka patches, all were his people. His were the canoes drawn up on the sand and his the empty houses where the war canoes had once rested on their rollers. Then as he cast his eyes from the lagoon to the canoe houses his brow contracted, and, turning his back to the la

In [ ]:
#delete datat
collection.delete(ids=["v_20250628"])


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionDeleteEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
#delete collection
chroma_client.delete_collection(name="spinned-books")